In [ ]:
%pip install gymnasium
%pip install numpy

In [4]:
from typing import Optional
import numpy as np
import gymnasium as gym
from gymnasium import spaces

## Base Env

In [10]:
class STBaseEnv(gym.Env):
  def __init__(self, data, ibalance=1e4):
    super(STBaseEnv, self).__init__()
    self.data = data
    self.ibalance = ibalance
    self.action_space = spaces.Discrete(3)
    self.balance = ibalance

    # data.shape[1] is the num of features, 2 is for balance and position
    self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1]+2))
    self.reset()

  def reset(self):
    self.balance = self.ibalance
    self.position = 0
    self.current_step =0

    return self._get_observation()

  def step(self, action):
    obs = self._get_observation()
    reward = 0  # calculate reward
    done = False
    info = {}

    return obs, reward, done, info

    def _get_observation(self):
      obs = np.concatenate([self.data[self.current_step], [self.balance, self.position]])
      return obs


## Wrappers

In [11]:
class MAIndicatorWrapper(gym.ObservationWrapper):
  def __init__(self, env, window=5):
    super(MAIndicatorWrapper, self).__init__(env)
    self.window = window

  def observation(self, observation):
    ma = self.env.data[max(0, self.env.current_step-self.window+1):self.env.current_step+1, 0].mean()
    return np.append(observation, ma)